# IDS2022 Project

In [1]:
# importing 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("clean_data.csv")

In [3]:
# data.head()
# ToimKell into 1 hour time intervsls
data["ToimKell"] = pd.to_datetime(data["ToimKell"], format='%H:%M')
data["ToimKell"] = data["ToimKell"].dt.hour
data.head()

,ToimKpv,ToimKell,ToimNadalapaev,SyndmusLiik,Kahjusumma,KohtLiik,MaakondNimetus,ValdLinnNimetus,KohtNimetus,Lest_X,Lest_Y,SyyteoLiik
0,2021-12-31,23,Reede,PISIVARGUS,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Mustamäe linnaosa,6586000-6586499,538500-538999,VT
1,2021-12-31,18,Reede,VARGUS,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Nõmme linnaosa,6583500-6583999,540500-540999,KT
2,2021-12-31,18,Reede,VARGUS,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Nõmme linnaosa,6583500-6583999,540500-540999,KT
3,2021-12-31,16,Reede,VARGUS,0-499,"AVALIK_KOHT,TANAV_VALJAK",Harju maakond,Tallinn,Kesklinna linnaosa,6588500-6588999,542500-542999,KT
4,2021-12-31,15,Reede,VARGUS,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Lasnamäe linnaosa,6589500-6589999,548500-548999,KT


In [4]:
# ToimKpv to separate year, month and date
data["ToimKpv"] = pd.to_datetime(data["ToimKpv"])
data['Aasta'] = pd.to_datetime(data["ToimKpv"]).dt.year
data['Kuu'] = pd.to_datetime(data["ToimKpv"]).dt.month
data['Kuupaev'] = pd.to_datetime(data["ToimKpv"]).dt.day
data.head()

,ToimKpv,ToimKell,ToimNadalapaev,SyndmusLiik,Kahjusumma,KohtLiik,MaakondNimetus,ValdLinnNimetus,KohtNimetus,Lest_X,Lest_Y,SyyteoLiik,Aasta,Kuu,Kuupaev
0,2021-12-31,23,Reede,PISIVARGUS,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Mustamäe linnaosa,6586000-6586499,538500-538999,VT,2021,12,31
1,2021-12-31,18,Reede,VARGUS,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Nõmme linnaosa,6583500-6583999,540500-540999,KT,2021,12,31
2,2021-12-31,18,Reede,VARGUS,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Nõmme linnaosa,6583500-6583999,540500-540999,KT,2021,12,31
3,2021-12-31,16,Reede,VARGUS,0-499,"AVALIK_KOHT,TANAV_VALJAK",Harju maakond,Tallinn,Kesklinna linnaosa,6588500-6588999,542500-542999,KT,2021,12,31
4,2021-12-31,15,Reede,VARGUS,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Lasnamäe linnaosa,6589500-6589999,548500-548999,KT,2021,12,31


In [5]:
# KohtLiik remove "AVALIK_KOHT"  (except when it's the only value)
data['KohtLiik'] = data['KohtLiik'].str.replace('AVALIK_KOHT,', '')
data['KohtLiik'] = data['KohtLiik'].str.replace('AVALIK_KOHT', '')
data['KohtLiik'] = data['KohtLiik'].replace('',np.nan,regex = True) # https://sparkbyexamples.com/pandas/pandas-replace-blank-values-with-nan/
data['KohtLiik'] = data['KohtLiik'].fillna("AVALIK_KOHT")
data['KohtLiik'].unique()

array(['KAUPLUS', 'TANAV_VALJAK', 'PARKLA,TANAV_VALJAK', 'TANKLA',
       'KAUBAMAJA', 'AVALIK_KOHT', 'PARKLA', 'TREPIKODA_KORIDOR',
       'HOTELL_MOTELL', 'KAUPLUS,REISILAEV', 'BUSSIPEATUS', 'SOOGIKOHT',
       'RAVIASUTUS', 'TAKSO', 'SADAM', 'UHISSOIDUK', 'LENNUJAAM',
       'SPORDIRAJATIS', 'KAUPLUS,KIOSK', 'MUU RUUM', 'PARK', 'KALMISTU',
       'OPPEASUTUS_SELLE_TERRITOORIUM', 'TANAV_VALJAK,UHISSOIDUK',
       'BUSSIJAAM_BUSSITERMINAAL', 'KAUPLUS,SOOGIKOHT,UHISSOIDUK',
       'MAANTEE', 'MUU KOHT', 'TURG', 'REISILAEV', 'APTEEK,',
       'OUEALA_LAHTINE_HOOV', 'PANGAAUTOMAAT', 'MUU KOHT,MUU RUUM',
       'OOKLUBI_DISKOTEEK', 'KAUBAMAJA,KINO', 'KAUPLUS,TANKLA',
       'KAUBAMAJA,KIOSK', 'KINO', 'SOOGIKOHT,TEENINDUSETTEVOTE',
       'TEENINDUSETTEVOTE', 'MUU KOHT,TOOTMIS_TOORUUM',
       'KAUBAMAJA,KAUPLUS', 'KASIINO', 'TAKSO,TANAV_VALJAK,UHISSOIDUK',
       'SOOGIKOHT,OOKLUBI_DISKOTEEK', 'KIRIK_MUU_KULTUSEHOONE',
       'SADAM,TANAV_VALJAK', 'TANAV_VALJAK,OOKLUBI_DISKOTEEK',
       

In [ ]:
# data kohtliik – kõigile jätta alles levinuim
mingid = data['KohtLiik'].tolist()
mingid2 = ",".join(mingid).split(',')
# print(mingid2)
from collections import Counter
c = Counter(mingid2)
#print( c.items() )
K = pd.DataFrame(c.items())
for i in range(len(data)):
    sone = data['KohtLiik'][i] 
    sone2 = sone.split(',')
    if (len(sone2)>1):
        L = []
        for j in range(len(sone2)):
            mitu = K.loc[K[0]==sone2[j]].values.flatten().tolist()[1]
            #print(mitu)
            L.append(mitu)
        #print(L)
        maks = max(L)
        indeks = L.index(maks)
        print(sone2[indeks])
        #data['KohtLiik'][i] = sone2[indeks]
        
    #data['Kohtliik'][i] = data['Kohtliik'][i] 

In [7]:
# ma tahan teada kas üldse töötab seega mingi mudel
datam1 = data.drop(["ToimKpv","Kahjusumma", "KohtLiik", "KohtNimetus","Lest_X", "Lest_Y" ], axis = 1)
datam1.SyyteoLiik = data.SyyteoLiik.map(dict(KT=1, VT=0))
data_dum = pd.get_dummies(datam1, columns =  ['ToimNadalapaev', 'SyndmusLiik', 'MaakondNimetus', 'ValdLinnNimetus'])
# data_dum.head()
labels = data_dum.SyyteoLiik
features = data_dum.drop('SyyteoLiik', axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size = 0.75, random_state=1)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=200,criterion='gini', random_state=0).fit(X_train, y_train)
#model = KNeighborsClassifier(n_neighbors = 2, metric= "manhattan").fit(X_train, y_train)
#dt = DecisionTreeClassifier(criterion='entropy', random_state=0).fit(X_train, y_train)

results_df = pd.DataFrame()
rf.fit(X_train, y_train)
acc = accuracy_score(y_test, rf.predict(X_test))
results_df = results_df.append({'model': 'RandomForest', 'accuracy': acc}, ignore_index=True)
print(results_df)


          model  accuracy
0  RandomForest  0.911572


/var/folders/dy/p_hwdm556d3d4y6m5yh6xcjh0000gn/T/ipykernel_15051/1382312094.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'model': 'RandomForest', 'accuracy': acc}, ignore_index=True)


In [9]:
# Only with date and time
#def MSE(y_target, y_pred):
#    return 1/len(y_target)* sum((y_target-y_pred)**2)
 
datam2 = data.drop(["ToimKpv","Kahjusumma", "KohtLiik", "KohtNimetus","Lest_X", "Lest_Y",'SyndmusLiik','MaakondNimetus', 'ValdLinnNimetus' ], axis = 1)
datam2.SyyteoLiik = data.SyyteoLiik.map(dict(KT=1, VT=0))
data_dum2 = pd.get_dummies(datam2, columns =  ['ToimNadalapaev'])
# data_dum.head()
labels = data_dum2.SyyteoLiik
features = data_dum2.drop('SyyteoLiik', axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size = 0.75, random_state=1)

rf2 = RandomForestClassifier(n_estimators=200,criterion='gini', random_state=0).fit(X_train, y_train)
rf2.fit(X_train, y_train)
acc = accuracy_score(y_test, rf2.predict(X_test))
results_df = results_df.append({'model': 'RandomForest2', 'accuracy': acc}, ignore_index=True)
print(results_df)



           model  accuracy
0   RandomForest  0.911572
1  RandomForest2  0.543483


/var/folders/dy/p_hwdm556d3d4y6m5yh6xcjh0000gn/T/ipykernel_15051/2234461037.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'model': 'RandomForest2', 'accuracy': acc}, ignore_index=True)


In [ ]:
# Model only with location
datam3 = data.drop(["ToimKpv","Kahjusumma","Lest_X", "Lest_Y",'SyndmusLiik','ToimNadalapaev'], axis = 1)
datam3.SyyteoLiik = data.SyyteoLiik.map(dict(KT=1, VT=0))
data_dum3 = pd.get_dummies(datam3, columns =  ["KohtLiik", "KohtNimetus", 'ValdLinnNimetus',  'MaakondNimetus'])
# data_dum.head()
labels = data_dum3.SyyteoLiik
features = data_dum3.drop('SyyteoLiik', axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size = 0.75, random_state=1)

rf3 = RandomForestClassifier(n_estimators=200,criterion='gini', random_state=0).fit(X_train, y_train)
rf3.fit(X_train, y_train)
acc = accuracy_score(y_test, rf3.predict(X_test))
results_df = results_df.append({'model': 'RandomForest3', 'accuracy': acc}, ignore_index=True)
print(results_df)

In [ ]:
datam2 = data.drop(["ToimKpv","Kahjusumma", "KohtLiik", "KohtNimetus","Lest_X", "Lest_Y" ], axis = 1)
datam2.SyyteoLiik = data.SyyteoLiik.map(dict(KT=1, VT=0))
data_dum2 = pd.get_dummies(datam2, columns =  ['ToimNadalapaev', 'SyndmusLiik', 'MaakondNimetus', 'ValdLinnNimetus'])
# data_dum.head()
labels = data_dum2.SyyteoLiik
features = data_dum2.drop('SyyteoLiik', axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size = 0.75, random_state=1)

rf2 = RandomForestClassifier(n_estimators=200,criterion='gini', random_state=0).fit(X_train, y_train)
rf2.fit(X_train, y_train)
acc = accuracy_score(y_test, rf2.predict(X_test))
results_df = results_df.append({'model': 'RandomForest2', 'accuracy': acc}, ignore_index=True)
print(results_df)

In [ ]:
def MSE(y_target, y_pred):
    return 1/len(y_target)* sum((y_target-y_pred)**2)
#print(MSE(y_test, rf2.predict(X_test)))
print(MSE(y_test, rf2.predict(X_test)))